**Идея текущего подзода:**

# Import

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../')

In [2]:
from src.config import (
    path_users_temp,
    path_persons_temp,
    path_sessions_train,
    path_sessions_test,
    path_sessions_val
)

# Чтение файлов

### Данные о пользователях

In [3]:
users = pd.read_csv(path_users_temp)
print(users.shape)
users.head()

(45167, 9)


,user_id,timestamp,age,gender,country,playcount,playlists,user_name,subscribertype
0,1,1116715959,24.0,f,US,221012.0,2.0,000123,base
1,2,1163123792,39.0,m,CZ,217535.0,9.0,000333,base
2,3,1184426573,NaN,f,NaN,49733.0,2.0,00elen,base
3,4,1123157597,32.0,m,DE,168054.0,2.0,00Eraser00,base
4,5,1171302116,23.0,m,UK,45700.0,2.0,00fieldsy,base


### Данные об исполнителях

In [4]:
# исполнители
persons = pd.read_csv(path_persons_temp)
print(persons.shape)
persons.head()

# поле "person_name" по фиксированному "person_id" может иметь несколько вариаций
# пример с "David Guetta"
# будем использовать для декодинга person_id
persons[persons['person_id'].isin([227])]

(595049, 3)


,person_id,person_name,person_MBID
4673,227,David+Guetta+&+Nicky+Romero,NaN
12079,227,"David+Guetta,+Sam+Martin",1bb1eec6-88c3-4028-8920-a985c4b9081a
17306,227,David+Guetta+ft.+Chris+Willis,1bb1eec6-88c3-4028-8920-a985c4b9081a
31403,227,David+Guetta+-+Ne-Yo+-+Kelly+Rowland,1bb1eec6-88c3-4028-8920-a985c4b9081a
38019,227,David+Guetta+&+Chris+Willis+Feat.+Fergie+&+LMFAO,1bb1eec6-88c3-4028-8920-a985c4b9081a
...,...,...,...
570347,227,David+Guetta+&+Alesso+feat.+Tegan+&+Sara,NaN
572905,227,David+Guetta+ft.+Kelly+Rowland,1bb1eec6-88c3-4028-8920-a985c4b9081a
578055,227,David+Guetta+Feat+Lil+Wayne+&+Chris+Brown,1bb1eec6-88c3-4028-8920-a985c4b9081a
588531,227,David+Guetta+&+Glowinthedark+feat.+Harrison+Shaw,NaN


### Данные о сессиях пользователей

In [5]:
%%time
sessions_train = pd.read_csv(path_sessions_train)
sessions_test = pd.read_csv(path_sessions_test)
# sessions_val = pd.read_csv(path_sessions_val)

CPU times: user 4.04 s, sys: 528 ms, total: 4.56 s
Wall time: 9.66 s


In [6]:
print(sessions_train.shape)
sessions_train.head()

(6549461, 10)


,session_id,timestamp,playtime,numtracks,user_id,track_id,track_playratio,playcount,person_id,numpersons
0,12,1405519516,5202,25,41504,1210840,1.0,353.0,154295,4
1,12,1405519516,5202,25,41504,1210840,1.0,94.0,154295,4
2,12,1405519516,5202,25,41504,1210766,1.0,1093.0,154295,4
3,12,1405519516,5202,25,41504,1210626,1.0,328.0,154295,4
4,12,1405519516,5202,25,41504,1210759,1.0,4.0,154295,4


In [7]:
print(sessions_test.shape)
sessions_test.head()

(1403506, 10)


,session_id,timestamp,playtime,numtracks,user_id,track_id,track_playratio,playcount,person_id,numpersons
0,20,1418303964,7654,31,41504,3488336,1.00,415.0,435780,4
1,20,1418303964,7654,31,41504,3488271,1.00,2395.0,435780,4
2,20,1418303964,7654,31,41504,3488314,1.00,1448.0,435780,4
3,20,1418303964,7654,31,41504,3488330,0.81,1806.0,435780,4
4,20,1418303964,7654,31,41504,1975962,1.46,76.0,247876,4


In [8]:
user_person_train = sessions_train[['session_id', 'user_id', 'person_id', 'track_playratio']]
user_person_test = sessions_test[['session_id', 'user_id', 'person_id', 'track_playratio']]

user_person_train.loc[user_person_train['track_playratio'] > 1, 'track_playratio'] = 1
user_person_train['track_playratio'] = user_person_train['track_playratio'].round()
# user_person_train = user_person_train[
#     ~user_person_train['track_playratio'].isna()
#     & ~(user_person_train['track_playratio'] == 0)
# ]

user_person_test.loc[user_person_test['track_playratio'] > 1, 'track_playratio'] = 1
user_person_test['track_playratio'] = user_person_test['track_playratio'].round()
# user_person_test = user_person_test[
#     ~user_person_test['track_playratio'].isna()
#     & ~(user_person_test['track_playratio'] == 0)
# ]

/home/user/miniconda3/envs/ds/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
<ipython-input-8-6c512d8e4b92>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_person_train['track_playratio'] = user_person_train['track_playratio'].round()
/home/user/miniconda3/envs/ds/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [9]:
user_person_train.head()

,session_id,user_id,person_id,track_playratio
0,12,41504,154295,1.0
1,12,41504,154295,1.0
2,12,41504,154295,1.0
3,12,41504,154295,1.0
4,12,41504,154295,1.0


In [10]:
user_person_test.head()

,session_id,user_id,person_id,track_playratio
0,20,41504,435780,1.0
1,20,41504,435780,1.0
2,20,41504,435780,1.0
3,20,41504,435780,1.0
4,20,41504,247876,1.0


In [11]:
def split_session_one(session):
    return [(session[session != person], person) for person in session]


def prepare_X_y(sessions_df): # le_user
    sessions_splited = sessions_df.copy()
    sessions_splited = sessions_splited.groupby(['session_id', 'user_id'])['person_id'].unique()
    sessions_splited = sessions_splited.apply(split_session_one)
    sessions_splited = sessions_splited.explode()
    sessions_splited = pd.DataFrame(
        sessions_splited.tolist(),
        columns=['X', 'y'],
        index=sessions_splited.index
    )
    return sessions_splited['X'], sessions_splited['y']

In [12]:
%%time
X_train, y_train = prepare_X_y(user_person_train)
X_test, y_test = prepare_X_y(user_person_test)

CPU times: user 38.9 s, sys: 1.13 s, total: 40.1 s
Wall time: 39.7 s


In [13]:
X_train

session_id  user_id
12          41504                               [288626, 341684, 325050]
            41504                               [154295, 341684, 325050]
            41504                               [154295, 288626, 325050]
            41504                               [154295, 288626, 341684]
69          40796                                [28445, 377440, 303680]
                                             ...                        
2764449     39905      [49737, 191960, 276125, 238733, 165188, 191300...
            39905      [49737, 191960, 276125, 238733, 165188, 191300...
            39905      [49737, 191960, 276125, 238733, 165188, 191300...
            39905      [49737, 191960, 276125, 238733, 165188, 191300...
            39905      [49737, 191960, 276125, 238733, 165188, 191300...
Name: X, Length: 3398732, dtype: object

In [14]:
# def unique_concat(x):
#     return np.unique(np.concatenate(x.values))

# X_cp = (X_train
#     .reset_index()
#     .groupby(['session_id', 'user_id'])['X']
#     .apply(unique_concat)
#     .explode()
#     .reset_index()
#     .rename(columns={'X':'person_id'})
#     .assign(track_playratio=1.0)
# )

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.sparse import csc_matrix, csr_matrix
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cdist


def hit_rate(y, y_pred):
    hits = [
        int(y_el in y_pred_el)
        for y_el, y_pred_el in zip(y, y_pred)
    ]
    return sum(hits) / len(hits)


def unique_concat(x):
    return np.unique(np.concatenate(x.values))


class SVDNearestPersons(BaseEstimator, ClassifierMixin):
    def __init__(self, num_top_elements=10):
        self.num_top_elements = num_top_elements
        self.le_user = LabelEncoder()
        self.le_person = LabelEncoder()
        self.X_cp_csc = None
        self.U = None
        self.sigma = None
        self.Vt = None
        self.V = None
        
    def fit(self, X):
        X_cp = (X
            .reset_index()
            .groupby(['session_id', 'user_id'])['X']
            .apply(unique_concat)
            .explode()
            .reset_index()
            .rename(columns={'X':'person_id'})
            .assign(track_playratio=1.0)
        )

        X_cp['user_id'] = self.le_user.fit_transform(X_cp['user_id'])
        X_cp['person_id'] = self.le_person.fit_transform(X_cp['person_id'])

        X_cp = X_cp.groupby(['user_id', 'person_id'], as_index=False)['track_playratio'].sum()
        
        # формат: csc_matrix((data, (row, col)))
        # взято отсюда: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html
        X_cp_csc = csc_matrix((X_cp['track_playratio'], (X_cp['user_id'], X_cp['person_id'])))
        self.X_cp_csc = X_cp_csc
        
        self.U, self.sigma, self.Vt = svds(
            X_cp_csc, #  - X_cp_csc.mean(axis=1)
            k = 100
        )
        self.sigma = np.diag(self.sigma)
        self.V = self.Vt.T
        
#     def predict(self, X):
#         X_cp = X.apply(self.le_person.transform)
#         y_pred = []
#         for (session_id, user_id), x in tqdm(zip(X_cp.index, X_cp)):
#             x_embedding = self.V[x]
#             dist = cdist(self.V, x_embedding, metric='cosine')
#             dist = dist.mean(axis=1)
#             #
#             top_n = dist.argsort()
#             # выбрасываем из топа person_id, которые были в x
#             # после выбиараем top_n 
#             top_n = top_n[~np.isin(top_n, x)][:self.num_top_elements]
#             # привод к настоящим person_id 
#             top_n = self.le_person.inverse_transform(top_n)
#             y_pred.append(top_n)
#         return np.array(y_pred)

    def predict(self, X):
        X_cp = X.apply(self.le_person.transform)
        y_pred = []
        for (session_id, user_id), x in tqdm(zip(X_cp.index, X_cp)):
            if user_id in self.le_user.classes_:
                user_id_transformed = self.le_user.transform([user_id])[0]
                user_raitings = np.dot(
                    np.dot(
                        self.U[[user_id_transformed]],
                        self.sigma),
                    self.Vt
                ).flatten()
                top_n = user_raitings.argsort()
                top_n = top_n[~np.isin(top_n, x)][-self.num_top_elements:]
            else:
                x_embedding = self.V[x]
                dist = cdist(self.V, x_embedding, metric='cosine')
                dist = dist.mean(axis=1) # min
                #
                top_n = dist.argsort()
                # выбрасываем из топа person_id, которые были в x
                # после выбиараем top_n 
                top_n = top_n[~np.isin(top_n, x)][:self.num_top_elements]
            # привод к настоящим person_id 
            top_n = self.le_person.inverse_transform(top_n)
            y_pred.append(top_n)
        return np.array(y_pred)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return hit_rate(y, y_pred)

In [16]:
%%time
svd_np = SVDNearestPersons()
svd_np.fit(X_train)

CPU times: user 30.8 s, sys: 591 ms, total: 31.4 s
Wall time: 21.9 s


In [17]:
X, y = X_test.head(1000), y_test.head(1000)

In [18]:
svd_np.score(X, y)

1000it [00:01, 591.75it/s]


0.132

# DRAFT

In [9]:
# from scipy.sparse import csc_matrix, csr_matrix
# from scipy.sparse.linalg import svds

# from sklearn.preprocessing import LabelEncoder


# le_user = LabelEncoder()
# le_person = LabelEncoder()

# user_person_train.loc[user_person_train['track_playratio'] > 1, 'track_playratio'] = 1
# user_person_train['track_playratio'] = user_person_train['track_playratio'].round()

# user_person_train['user_id'] = le_user.fit_transform(user_person_train['user_id'])
# user_person_train['person_id'] = le_person.fit_transform(user_person_train['person_id'])

# user_person_train = user_person_train.groupby(['user_id', 'person_id'], as_index=False)['track_playratio'].sum()

# # формат: csc_matrix((data, (row, col))), но работает: csc_matrix((data, (col, row)))
# # взято отсюда: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html
# user_person_csc = csc_matrix(
#     (user_person_train['track_playratio'], (user_person_train['user_id'], user_person_train['person_id']))
# )
# user_person_csc

# U, sigma, Vt = svds(user_person_csc, k = 100)
# sigma = np.diag(sigma)
# print(U.shape)
# print(sigma.shape)
# print(Vt.shape)

# # all_user_predicted_ratings = 
# np.dot(np.dot(U, sigma), Vt).shape #+ user_ratings_mean.reshape(-1, 1)

In [24]:
# X_cp = X.apply(svd_np.le_person.transform)
# for (session_id, user_id), x in tqdm(zip(X_cp.index, X_cp)):
# #     if user_id in 
#     x_embedding = svd_np.V[x]
#     dist = cdist(svd_np.V, x_embedding, metric='cosine')
#     dist = dist.mean(axis=1) # min
#     #
#     top_n = dist.argsort()
#     # выбрасываем из топа person_id, которые были в x
#     # после выбиараем top_n 
#     top_n = top_n[~np.isin(top_n, x)][:svd_np.num_top_elements]
#     # привод к настоящим person_id 
#     top_n = svd_np.le_person.inverse_transform(top_n)
#     break

0it [00:00, ?it/s]


In [61]:
# if user_id in svd_np.le_user.classes_:
#     user_id_transformed = svd_np.le_user.transform([user_id])[0]
#     print(user_id, user_id_transformed)
#     user_raitings = np.dot(
#         np.dot(
#             svd_np.U[[user_id_transformed]],
#             svd_np.sigma),
#         svd_np.Vt
#     ).flatten()
#     top_n = user_raitings.argsort()
#     top_n = top_n[~np.isin(top_n, x)][-svd_np.num_top_elements:]

41504 27783
